In [26]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
import warnings
warnings.filterwarnings(action = 'ignore')



In [27]:
df = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울시 신호등 관련 정보.csv",encoding='CP949')
df = df.drop(['상태 (공통)', '부착방식', '부착대길이', '고가 (공통)', '부착대방향', '배면등수량',
       '신호등종류', '배면등종류', '설치일', '교체일', '지주관리번호', '신호발광구분', '제조회사', '작업구분 (공통)',
       '표출구분 (공통)', '신규정규화ID', '공사관리번호', '부착대관리번호', '이력ID', '위치정보',
       '공사형태 (공통)','부착대관리번호.1'],axis='columns')

df = df.rename({'X좌표':'위도','Y좌표':'경도'},axis='columns')
df

,신호등수량,위도,경도
0,2,202650.00560,542705.46253
1,3,202714.48048,542715.77569
2,1,203129.34784,541838.75856
3,1,202976.86616,542798.94598
4,1,203026.11371,542800.32686
...,...,...,...
109995,1,183933.70795,543097.23710
109996,1,206196.04866,559059.56161
109997,1,205062.84369,563479.41770
109998,1,205142.04368,563497.91769


In [28]:
def get_traffic_light(lon, lat, km):
    df = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울시 신호등 관련 정보.csv",encoding='CP949')
    
    df['X좌표'] = df['X좌표'].astype(float)
    df['Y좌표'] = df['Y좌표'].astype(float)
    df['geometry'] = df.apply(lambda row : Point([row['X좌표'], row['Y좌표']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry')
    df.crs = from_string("+proj=tmerc +lat_0=38 +lon_0=127 +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +units=m +no_defs")
    wgs84 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    df = df.to_crs(wgs84)
    df2 = pd.DataFrame()
    df2['lon'] = df['geometry'].map(lambda t : t.x)
    df2['lat'] = df['geometry'].map(lambda t : t.y)
    df2['신호등수량'] = df['신호등수량']
    df2 = df2.rename({'lat':'위도','lon':'경도'},axis='columns')
    df2 = df2[['신호등수량','위도','경도']]
    return get_xy(df2, lon, lat, km)

target_

In [36]:
def get_traffic_light(lon, lat, km):
    global df,df2
    df = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울시 신호등 관련 정보.csv",encoding='CP949')
    df['X좌표'] = df['X좌표'].astype(float)
    df['Y좌표'] = df['Y좌표'].astype(float)
    df['geometry'] = df.apply(lambda row : Point([row['X좌표'], row['Y좌표']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry')
    df.crs = from_string("+proj=tmerc +lat_0=38 +lon_0=127 +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +units=m +no_defs")
    wgs84 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    df = df.to_crs(wgs84)
    df['lon'] = df['geometry'].map(lambda t : t.x)
    df['lat'] = df['geometry'].map(lambda t : t.y)
#     df['신호등수량'] = df['신호등수량']
    df = df.rename({'lat':'위도','lon':'경도'},axis='columns')
    df = df[['신호등수량','위도','경도']]
    return get_xy(df, lon, lat, km)

target_tl = get_traffic_light(127.036042,37.516145,0.5)
target_tl

,신호등수량,위도,경도
919,2,37.515204,127.035544
920,1,37.515205,127.035537
1240,1,37.515372,127.035162
1241,1,37.515422,127.035137
1242,2,37.515284,127.035130
...,...,...,...
86590,1,37.515476,127.035098
89853,2,37.518193,127.035129
94405,1,37.515367,127.035712
98306,1,37.515163,127.035155


In [43]:
a = target_tl['신호등수량'].values.tolist()
sum(a)

137

In [30]:
lon =126.955009
lat =37.6027953
#tmp = get_tlsghemd(lon,lat,2)
#tmp

In [31]:
#위도 =111.195km , 경도 88.8km , 2km 박스일 때 반경  1.414
def dis_to_lat(km):
    return km/111.195
def dis_to_lon(km):
    return km/88.8

def lat_to_dis(lat):
    return lat*111.195
def lon_to_dis(lon):
    return lon*88.8

def get_xy(data,lon, lat,km):
    lat_size = dis_to_lat(km)
    lon_size = dis_to_lon(km)
    return data[(data['위도']<=lat+lat_size/2) & (data['위도']>=lat-lat_size/2) &
                 (data['경도']<=lon+lon_size/2) & (data['경도']>=lon-lon_size/2)]

In [32]:
df.head(5)

,신호등수량,위도,경도
0,2,37.483790,127.029962
1,3,37.483883,127.030691
2,1,37.475980,127.035378
3,1,37.484631,127.033658
4,1,37.484644,127.034215


In [33]:
df.columns

Index(['신호등수량', '위도', '경도'], dtype='object')

df = df.drop(['상태 (공통)', '부착방식', '부착대길이', '고가 (공통)', '부착대방향', '배면등수량',
       '신호등종류', '배면등종류', '설치일', '교체일', '지주관리번호', '신호발광구분', '제조회사', '작업구분 (공통)',
       '표출구분 (공통)', '신규정규화ID', '공사관리번호', '부착대관리번호', '이력ID', '위치정보',
       '공사형태 (공통)','부착대관리번호.1'],axis='columns')

In [34]:
df.head(5)

,신호등수량,위도,경도
0,2,37.483790,127.029962
1,3,37.483883,127.030691
2,1,37.475980,127.035378
3,1,37.484631,127.033658
4,1,37.484644,127.034215


In [35]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString


df['X좌표'] = df['X좌표'].astype(float)
df['Y좌표'] = df['Y좌표'].astype(float)
df['geometry'] = df.apply(lambda row : Point([row['X좌표'], row['Y좌표']]), axis=1)
df = gpd.GeoDataFrame(df, geometry='geometry')

KeyError: 'X좌표'

In [ ]:
from fiona.crs import from_string

epsg5186 = from_string("+proj=tmerc +lat_0=38 +lon_0=127 +k=1 +x_0=200000 +y_0=600000 +ellps=GRS80 +units=m +no_defs")

In [ ]:
df.crs = epsg5186
print(df.crs)

In [ ]:
wgs84 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [ ]:
df = df.to_crs(wgs84)
df["geometry"].head()

In [ ]:
df.head()

In [ ]:
df.iloc[1]['geometry'].x
df2 = pd.DataFrame()
df2['lon'] = df['geometry'].map(lambda t : t.x)
df2['lat'] = df['geometry'].map(lambda t : t.y)
df2['신호등수량'] = df['신호등수량']

In [ ]:
df2

In [ ]:
df2 = df2.rename({'lat':'위도','lon':'경도'},axis='columns')
df2 = df2[['신호등수량','위도','경도']]

In [ ]:
lon =126.955009
lat =37.6027953

tmp1 = get_xy(df2, lon, lat, 2) # return dataframe 
tmp2 = get_xy(df2, lon, lat, 1)

print(tmp1,tmp1['신호등수량'].sum())
print(tmp2,tmp2['신호등수량'].sum())

In [ ]:
def get_crosswalk(lon,lat,km) :
    df3 = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울시 횡단보도 위치정보 (좌표계_ WGS1984).csv",encoding='CP949')
    df3 = df3.drop(['순번', '상태', '횡단보도종류코드', '가로길이', '세로길이', '화살표시수량', '화살표시길이',
       '고가', '구경찰서코드', '구코드', '동코드', '지번', '신경찰서코드', '작업구분', '표출구분', '도로구분',
       '관할사업소', '신규정규화ID', '설치일', '교체일', '이력ID', '공사관리번호', '횡단보도관리번호.1',
       '공사형태'], axis='columns')
    return get_xy(df3, lon, lat, km)

In [ ]:
df3 = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울시 횡단보도 위치정보 (좌표계_ WGS1984).csv",encoding='CP949')

In [ ]:
df3.head()

In [ ]:
df3.columns

In [ ]:
df3 = df3.drop(['순번', '상태', '횡단보도종류코드', '가로길이', '세로길이', '화살표시수량', '화살표시길이',
       '고가', '구경찰서코드', '구코드', '동코드', '지번', '신경찰서코드', '작업구분', '표출구분', '도로구분',
       '관할사업소', '신규정규화ID', '설치일', '교체일', '이력ID', '공사관리번호', '횡단보도관리번호.1',
       '공사형태'], axis='columns')

In [ ]:
tmp3 = get_xy(df3, lon, lat, 2) # return dataframe 
tmp4 = get_xy(df3, lon, lat, 1)
print(tmp3,sep='\n')
print(tmp4,len(tmp4))

In [ ]:
def get_bus_stop(lon,lat,km) :
    df4 = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울특별시 버스정류소 위치정보.csv",encoding='CP949')
    df4 = df4.drop(['정류소번호'],axis=1)
    df4 = df4.rename({'X좌표':'경도','Y좌표':'위도'},axis=1)
    df4 = df4[['정류소명','위도','경도']]
    
    return get_xy(df4,lon,lat,km)

In [ ]:
df4 = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/서울특별시 버스정류소 위치정보.csv",encoding='CP949')


In [ ]:
df4.head()

In [ ]:
df4 = df4.drop(['정류소번호'],axis=1)

In [ ]:
df4 = df4.rename({'X좌표':'경도','Y좌표':'위도'},axis=1)
df4 = df4[['정류소명','위도','경도']]

In [ ]:
tmp5 = get_xy(df4, lon, lat, 2) # return dataframe 
tmp6 = get_xy(df4, lon, lat, 1)
print(tmp5,sep='\n')
print(tmp6,sep='\n')
print("범위 내 정류장의 갯수는 %d개 입니다."%len(tmp6))

In [ ]:
def get_subway(lon,lat,km) :
    df5 = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/station_coordinate.csv")
    df5 = df5.drop(['line','code'],axis=1)
    df5 = df5.rename({'lat':'위도','lng':'경도'},axis=1)
    return get_xy(df5,lon,lat,km)

In [ ]:
df5 = pd.read_csv("/Users/chanju/github/smussp/6주차/이찬주/station_coordinate.csv")

In [ ]:
df5.head()

In [ ]:
df5.columns

In [ ]:
df5 = df5.drop(['line','code'],axis=1)
df5 = df5.rename({'lat':'위도','lng':'경도'},axis=1)
df5.head()

In [ ]:
tmp7 = get_xy(df5, lon, lat, 6) # return dataframe 
tmp7